In [1]:
#MNIST with CNN
from tensorflow.examples.tutorials.mnist import input_data
#텐서플로우에서 제공한 MNIst예제
import tensorflow as tf

tf.reset_default_graph()

#1. Data Loading
mnist = input_data.read_data_sets("./Data/Mnist", one_hot = True) 
#원래 폴더 없다면 만들어서 데이터 넣어준다. #one_hot y측 레이블 여러개

#2. Placeholder
X = tf.placeholder(shape = [None,784], dtype = tf.float32)
 #데이터 개수 미정, 컬럼 784개 (잘 알려진 예제이다. )
Y = tf.placeholder(shape = [None,10], dtype = tf.float32)
keep_rate = tf.placeholder(dtype = tf.float32) #스칼라이므로 모양 잡을 필요 없다. dropout비율

#3. Convolution Layer
##3.1 . Layer1 (여러개의 계층으로 구성할 수 있다. )
#이미지 데이터를 사차 배열로 바꿔야 함. (이미지 수, 가로, 세로, 색)
X_img = tf.reshape(X,shape = [-1,28,28,1]) #[,,,] 4차 배열
#텐서로 모양 바꾸면 텐서 타입으로 빠진다. #-1 나머지를 이미지 사이즈로 만들어. 데이터 개수를 계산하지 않을거야

#    filter생성
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev = 0.01)) #3*3크기, 깊이 1 흑백, 서른 두개의 필터#표준편차 설정 비슷한 값 도출
#    Convolution
L1 = tf.nn.conv2d(X_img, W1, strides = [1,1,1,1], padding = "SAME") #1*1이고 패딩 작업
#    ReLU
L1 = tf.nn.relu(L1)
#    Max Pooling
L1 = tf.nn.max_pool(L1, ksize= [1,2,2,1],strides = [1,2,2,1], padding ="SAME" )

##3.1 . Layer1 
#    Filter,Convolution,ReLU
L2 = tf.layers.conv2d(inputs= L1, filters = 64, kernel_size = [3,3],  #패키지가 다름. #ksize정수로 떨어지도록 계산해야한다. 
                     padding ="SAME", strides = 1, activation = tf.nn.relu)
#Max pooling
L2 = tf.layers.max_pooling2d(inputs=L2, pool_size = [2,2], padding="SAME", strides = 2) #ksize와 같은 내용 다른 표현 psize
#이미지 크기 반으로 줄어든다. 

##시간을 절약하려면 dropout 하면 된다. 

###end Convolution Layer

#4. FC(Neural Network)
#데이터에 대한 2차 배열 이미지로 바꾸어야 한다. 
L2 = tf.reshape(L2, shape = [-1, 7*7*64]) #L2.shape(?,7,7,64)

#5. Weight & bias
W2 = tf.get_variable("weight2", shape= [7*7*64,256],initializer = tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]), name="bias2")

_layer1 = tf.nn.relu(tf.matmul(L2,W2) + b2)
layer1 = tf.nn.dropout(_layer1, keep_prob = keep_rate)

W3 = tf.get_variable("weight3", shape= [256,256],initializer = tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([256]), name="bias3")

_layer2 = tf.nn.relu(tf.matmul(layer1,W3) + b3)
layer2 = tf.nn.dropout(_layer2, keep_prob = keep_rate)

W4 = tf.get_variable("weight4", shape= [256,10],initializer = tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([10]), name="bias4")

#6. Hypothesis
H = tf.matmul(layer2,W4) + b4

#7. Cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = H, labels= Y))

#8. Train
train = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)

#9. Session, 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#10. 학습 
#gpu 쓸 때는 리소스 때문에 배치를 해야한다. 

num_of_epoch = 10
batch_size = 100

for step in range(num_of_epoch):
    num_of_iter = int(mnist.train.num_examples/batch_size)
    
    for i in range(num_of_iter):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        _, cost_val = sess.run([train, cost], feed_dict={X:batch_x, Y:batch_y, keep_rate:0.5}) 
        #0.25 노드 많을 때, 0.7도 사용
    
    print("Cost :{}".format(cost_val))
    
#accuracy
predict = tf.argmax(H,1)
correct = tf.equal(predict,tf.argmax(Y,1))

accuracy = tf.reduce_mean(tf.cast(correct, dtype = tf.float32))

print("정확도 :{}".format(sess.run(accuracy, feed_dict = {X: mnist.test.images, Y: mnist.test.labels, keep_rate:1})))



Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./Data/Mnist\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./Data/Mnist\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./Data/Mnist\t10k-images-idx3-ubyte.gz
Extracting ./Data/Mnist\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Cost :0.125859797000885
Cost :0.18367843329906464
Cost :0.06371414661407471
Cost :0.031645841896533966
Cost :0.14998455345630646
Cost :0.09627361595630646
Cost :0.11077354103326797
Cost :0.011083091609179974
Cost :0.0350564680993557
Cost :0.0011081480188295245
정확도 :0.9912999868392944
